# LSTM

## Load imports.

In [1]:
# Make common scripts visible
import sys
sys.path.append('../common/')

from gensim.models import KeyedVectors
import numpy as np

from loader import load_preprocessed_data
from word_embeddings import DocToIntSequenceConverter
from lstm import LstmPredictor
from lookup_tables import int_to_topic_code, topic_code_to_topic_dict, topic_code_to_int
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

Using TensorFlow backend.


## Load the data

In [2]:
x, y = load_preprocessed_data('data/rcv1_no_stopwords_reduced.csv')
x = np.array(x)
y = np.array(y)

# Split data into 60% train, 20% validation, 20% test
total_examples = len(y)

split_point_1 = int(total_examples * 0.6)
split_point_2 = int(total_examples * 0.8)

train_x = x[:split_point_1]
train_y = y[:split_point_1]

val_x = x[split_point_1:split_point_2]
val_y = y[split_point_1:split_point_2]

test_x = x[split_point_2:]
test_y = y[split_point_2:]

print('Number of training examples: {}'.format(len(train_x)))

Number of training examples: 1568


## Pre-process the data

In [3]:
# Find the length of a tweet in words
article_lengths = np.array([len(article.split()) for article in train_x])

print('Minimum length of article in words: {}'.format(np.min(article_lengths)))
print('Maximum length of article in words: {}'.format(np.max(article_lengths)))
print('Mean length of article in words: {:.4f}'.format(np.mean(article_lengths)))
print('St dev of length of article in words: {:.4f}'.format(np.std(article_lengths)))

# Set the max sequence length to mean plus 3 standard deviations (99.7% confidence)
max_sequence_length = int(np.mean(article_lengths) + np.std(article_lengths)*3)

# Confirm not many tweets exceed this limit
articles_exceeding_limit = [article for article in train_x if len(article.split()) > max_sequence_length]
percentage_articles_exceeding_limit = (len(articles_exceeding_limit)/len(train_x))*100
print('Percentage of articles exceeding max sequence length limit: {:.4f}%'.format(percentage_articles_exceeding_limit))

Minimum length of article in words: 7
Maximum length of article in words: 519
Mean length of article in words: 112.1582
St dev of length of article in words: 72.5444
Percentage of articles exceeding max sequence length limit: 0.3827%


In [4]:
# Convert articles to sequence of integers representing the words
article_to_int_seq_converter = DocToIntSequenceConverter(x, max_sequence_length)
train_x_seq = article_to_int_seq_converter.convert_to_integer_sequences(train_x)
val_x_seq = article_to_int_seq_converter.convert_to_integer_sequences(val_x)
test_x_seq = article_to_int_seq_converter.convert_to_integer_sequences(test_x)

## Create the LSTM and train it.

In [5]:
word_embedding_dim = 300
word2vec_model = KeyedVectors.load_word2vec_format('../tweet-classification/embeddings/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [7]:
lstm = LstmPredictor(article_to_int_seq_converter.get_word_index(),
                     word_embedding_dim,
                     max_sequence_length,
                     word2vec_model,
                     len(int_to_topic_code.values()))
class_weights = compute_class_weight('balanced', np.unique(train_y), train_y)
class_weights_dict = {}
for i in range(len(class_weights)):
    class_weights_dict[i] = class_weights[i]
lstm.train(train_x_seq, train_y, val_x_seq, val_y, class_weight=class_weights)

Train on 1568 samples, validate on 523 samples
Epoch 1/100
1568/1568 [==============================] - 50s 32ms/step - loss: 1.3346 - acc: 0.5402 - val_loss: 1.0872 - val_acc: 0.6348
Epoch 2/100
1568/1568 [==============================] - 44s 28ms/step - loss: 0.7981 - acc: 0.7073 - val_loss: 0.7209 - val_acc: 0.7400
Epoch 3/100
1568/1568 [==============================] - 44s 28ms/step - loss: 0.5157 - acc: 0.8202 - val_loss: 0.5349 - val_acc: 0.8528
Epoch 4/100
1568/1568 [==============================] - 43s 28ms/step - loss: 0.3981 - acc: 0.8788 - val_loss: 0.3882 - val_acc: 0.8948
Epoch 5/100
1568/1568 [==============================] - 44s 28ms/step - loss: 0.3449 - acc: 0.9005 - val_loss: 0.7061 - val_acc: 0.7725
Epoch 6/100
1568/1568 [==============================] - 47s 30ms/step - loss: 0.4102 - acc: 0.8807 - val_loss: 0.3472 - val_acc: 0.9006
Epoch 7/100
1568/1568 [==============================] - 45s 29ms/step - loss: 0.2522 - acc: 0.9279 - val_loss: 0.3612 - val_acc: 0

1568/1568 [==============================] - 43s 27ms/step - loss: 0.0442 - acc: 0.9885 - val_loss: 0.2519 - val_acc: 0.9331
Epoch 61/100
1568/1568 [==============================] - 43s 27ms/step - loss: 0.1349 - acc: 0.9534 - val_loss: 0.3082 - val_acc: 0.9101


## Make predictions and report classification accuracy.

In [8]:
# Re-initialise the LSTM, will use weights from the previous training run.
lstm = LstmPredictor(article_to_int_seq_converter.get_word_index(),
                     word_embedding_dim,
                     max_sequence_length,
                     word2vec_model,
                     len(int_to_topic_code.values()),
                     use_saved_weights=True)
test_y_predict = lstm.predict(test_x_seq)
print(classification_report(test_y, test_y_predict, digits=6, target_names=topic_code_to_topic_dict.values()))

                        precision    recall  f1-score   support

CRIME, LAW ENFORCEMENT   0.957895  0.952880  0.955381       191
  ECONOMIC PERFORMANCE   0.933333  0.965517  0.949153        58
             ELECTIONS   0.866667  0.945455  0.904348        55
                HEALTH   0.903226  0.777778  0.835821        36
              RELIGION   0.916667  1.000000  0.956522        11
                SPORTS   1.000000  0.988372  0.994152       172

             micro avg   0.954111  0.954111  0.954111       523
             macro avg   0.929631  0.938334  0.932563       523
          weighted avg   0.954794  0.954111  0.953868       523



## Find examples where predictions went wrong

In [9]:
for topic_code, index in topic_code_to_int.items():
    topic_subset = test_y_predict[test_y == index]
    topic_subset_incorrect = topic_subset[topic_subset != index]
    document_subset = test_x[test_y == index]
    document_subset = document_subset[topic_subset != index]
    
    print('------ 5 random erroneous predictions for {} ------'.format(topic_code_to_topic_dict[topic_code]))
    print('')
    random_indices = np.random.choice(np.arange(len(topic_subset_incorrect)), 5)
    for index in random_indices:
        print(document_subset[index])
        print('')
        print('Above classified as {}'.format(topic_code_to_topic_dict[int_to_topic_code[topic_subset_incorrect[index]]]))
        print('')
    print('')

------ 5 random erroneous predictions for CRIME, LAW ENFORCEMENT ------

whole world Chicago police crowds anti-war demonstrators truncheons skulls protesters reporters bystanders years Chicago police black-and-white TV footage colleagues Democratic National_Convention President_Bill_Clinton world accomodating police department police spokesman Paul_Jenkins attitudinal change part democratic process officers TV radio Times demonstrators heirs thousands protesters Chicago opposition Vietnam_War revolution-minded leaders Tom_Hayden founder Students Democratic Society Abbie_Hoffman rambunctious Youth_International_Party Yippies Hayden California delegate convention Hoffman dead many yippies yuppies forces turbulent 1960s Vietnam_War military draft black civil rights exuberant youthful devotion sex drugs rock n roll time left identity politics oppressed author David_Horowitz self-described former-revolutionary-turned-Republican process people way demonstrators Aug. Democratic convention tw

ValueError: a must be non-empty